In [1]:
# libraries
import requests
import pandas as pd
import sqlite3

pd.set_option('display.max_info_columns', 1000)
pd.set_option('display.max_info_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

C:\Users\tito\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [11]:
# pulling team and player data
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
data = requests.get(url).json()

players = pd.DataFrame(data['elements'])
teams = pd.DataFrame(data['teams'])

In [12]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 101 columns):
 #    Column                                Non-Null Count  Dtype  
---   ------                                --------------  -----  
 0    can_transact                          696 non-null    bool   
 1    can_select                            696 non-null    bool   
 2    chance_of_playing_next_round          142 non-null    float64
 3    chance_of_playing_this_round          133 non-null    float64
 4    code                                  696 non-null    int64  
 5    cost_change_event                     696 non-null    int64  
 6    cost_change_event_fall                696 non-null    int64  
 7    cost_change_start                     696 non-null    int64  
 8    cost_change_start_fall                696 non-null    int64  
 9    dreamteam_count                       696 non-null    int64  
 10   element_type                          696 non-null    int64  
 11   ep_n

In [3]:
# pulling fixture data
fix_url = "https://fantasy.premierleague.com/api/fixtures/"
fixtures = requests.get(fix_url).json()
fixtures = pd.DataFrame(fixtures)

In [4]:
# cleaning player data
cols_to_keep_players = [
    "id", "first_name", "second_name", "element_type", "now_cost", 'form',
    "event_points", "selected_by_percent", "chance_of_playing_next_round", "team",
    "minutes", "goals_scored", "assists", "clean_sheets",
    "goals_conceded", "own_goals", "penalties_saved",
    "penalties_missed", "yellow_cards", "red_cards", "saves",
    "bonus", "bps", "influence", "creativity", "threat", "ict_index",
    "clearances_blocks_interceptions", "recoveries", "tackles",
    "defensive_contribution", "starts", "expected_goals",
    "expected_assists", "expected_goals_conceded"
]
player_df = players[cols_to_keep_players]

rename_dict = {"element_type": 'position',
               "now_cost": "current_cost",
               "id": "player_id",
               "event_points": "current_gw_points",
               "selected_by_percent": "current_selected_by_percentage"}
player_df = player_df.rename(rename_dict, axis=1)

player_df['player_name'] = player_df['first_name'] + " " + player_df['second_name']
player_df.drop(['first_name', 'second_name'], axis=1, inplace=True)

pos_replace_dict = {1: 'GK',
                    2: 'DEF',
                    3: 'MID',
                    4 : 'FWD'}
player_df['position'] = player_df['position'].replace(pos_replace_dict)

player_df['current_cost'] = player_df['current_cost']/10

player_df['chance_of_playing_next_round'] = player_df['chance_of_playing_next_round'].fillna(100)

player_df.head(5)

,player_id,position,current_cost,form,current_gw_points,current_selected_by_percentage,chance_of_playing_next_round,team,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,clearances_blocks_interceptions,recoveries,tackles,defensive_contribution,starts,expected_goals,expected_assists,expected_goals_conceded,player_name
0,1,GK,5.5,10.0,10,19.5,100.0,1,90,0,0,1,0,0,0,0,1,0,7,3,38,49.2,0.0,0.0,4.9,1,13,0,0,1,0.00,0.00,1.52,David Raya Martín
1,2,GK,4.5,0.0,0,0.9,100.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.00,0.00,0.00,Kepa Arrizabalaga Revuelta
2,3,GK,4.0,0.0,0,0.6,100.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.00,0.00,0.00,Karl Hein
3,4,GK,4.0,0.0,0,0.2,100.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.00,0.00,0.00,Tommy Setford
4,5,DEF,6.0,6.0,6,19.4,100.0,1,90,0,0,1,0,0,0,0,0,0,0,0,23,22.8,0.3,4.0,2.7,6,1,1,7,1,0.00,0.00,1.52,Gabriel dos Santos Magalhães


In [5]:
# running player points database
def point_history_updater(hist_file, gameweek):
    # Select & rename relevant columns
    df = player_df[['player_id', 'player_name', 'position', 
                    'current_cost', 'current_gw_points', 'current_selected_by_percentage']].copy()
    
    df = df.rename(columns={
        'current_cost': 'cost',
        'current_gw_points': 'gw_points',
        'current_selected_by_percentage': 'selected_by_percentage'
    })
    df['gw'] = gameweek
    
    try:
        hist_data = pd.read_csv(hist_file)
    except FileNotFoundError:

        df.to_csv(hist_file, index=False)
        return df
    
    merged = pd.concat([hist_data, df], ignore_index=True)
    merged = merged.drop_duplicates(subset=['player_id', 'gw'], keep='last')
    
    merged.to_csv(hist_file, index=False)
    return merged


In [6]:
player_points = point_history_updater(hist_file = "player_points.csv", gameweek=1)

In [7]:
# cleaning team data
cols_to_lose_teams = ["team_division", "unavailable", "pulse_id", "code"]
teams_df = teams.drop(cols_to_lose_teams, axis=1)
teams_df.rename({"id": "team_id"}, axis = 1, inplace=True)

teams_df.head()

,draw,form,team_id,loss,name,played,points,position,short_name,strength,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away
0,0,None,1,0,Arsenal,0,0,6,ARS,4,0,1320,1325,1350,1350,1290,1300
1,0,None,2,0,Aston Villa,0,0,9,AVL,3,0,1125,1250,1110,1200,1140,1300
2,0,None,3,0,Burnley,0,0,18,BUR,2,0,1050,1050,1050,1050,1050,1050
3,0,None,4,0,Bournemouth,0,0,16,BOU,3,0,1150,1180,1100,1160,1200,1200
4,0,None,5,0,Brentford,0,0,17,BRE,3,0,1120,1185,1080,1080,1160,1290


In [8]:
# cleaning fixtures data
fixtures = fixtures.drop(['stats'], axis=1)

fixtures['home_id'] = fixtures['team_h']
fixtures['away_id'] = fixtures['team_a']

except_cols = ['team_h', 'team_a']
fixtures = fixtures.melt(id_vars= [col for col in fixtures.columns if col not in except_cols], var_name='status', value_name='team_id')
fixtures.sort_values('pulse_id', inplace=True)

fixtures['status'] = fixtures['status'].replace({'team_a' : 'away', 'team_h' : 'home'})

fixtures['opponent_id'] = fixtures.apply(
    lambda x: x['away_id'] if x['status'] == 'home' else x['home_id'],
    axis=1
)


fixtures['team_difficulty'] = fixtures.apply(lambda x : x['team_h_difficulty'] if x['status'] == 'home' else x['team_a_difficulty'], axis=1)
fixtures['opponent_difficulty'] = fixtures.apply(lambda x : x['team_a_difficulty'] if x['status'] == 'home' else x['team_h_difficulty'], axis=1)
fixtures['team_score'] = fixtures.apply(lambda x : x['team_h_score'] if x['status'] == 'home' else x['team_a_score'], axis=1)
fixtures['opponent_score'] = fixtures.apply(lambda x : x['team_a_score'] if x['status'] == 'home' else x['team_h_score'], axis=1)

fixtures = fixtures.drop(['team_h_difficulty', 'team_a_difficulty', 'team_h_score', 'team_a_score', 'home_id','away_id'], axis=1)

fixtures.head(20)

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,pulse_id,status,team_id,opponent_id,team_difficulty,opponent_difficulty,team_score,opponent_score
0,2561895,1,True,True,1,2025-08-15T19:00:00Z,90,False,True,124791,away,4,12,5,3,2.0,4.0
380,2561895,1,True,True,1,2025-08-15T19:00:00Z,90,False,True,124791,home,12,4,3,5,4.0,2.0
381,2561896,1,True,True,2,2025-08-16T11:30:00Z,90,False,True,124792,home,2,15,3,4,0.0,0.0
1,2561896,1,True,True,2,2025-08-16T11:30:00Z,90,False,True,124792,away,15,2,4,3,0.0,0.0
2,2561897,1,True,True,3,2025-08-16T14:00:00Z,90,False,True,124793,away,10,6,3,3,1.0,1.0
382,2561897,1,True,True,3,2025-08-16T14:00:00Z,90,False,True,124793,home,6,10,3,3,1.0,1.0
387,2561898,1,True,True,4,2025-08-17T13:00:00Z,90,False,True,124794,home,16,5,3,3,3.0,1.0
7,2561898,1,True,True,4,2025-08-17T13:00:00Z,90,False,True,124794,away,5,16,3,3,1.0,3.0
4,2561899,1,True,True,5,2025-08-16T14:00:00Z,90,False,True,124795,away,19,17,2,2,0.0,3.0
384,2561899,1,True,True,5,2025-08-16T14:00:00Z,90,False,True,124795,home,17,19,2,2,3.0,0.0


In [ ]:
# SQL database
conn = sqlite3.connect("fpl.db")
player_df.to_sql("players", conn, if_exists="replace", index=False)
teams_df.to_sql("teams", conn, if_exists="replace", index=False)
fixtures.to_sql("fixtures", conn, if_exists="replace", index=False)
player_points.to_sql("player_points", conn, if_exists="replace", index=False)

696